# Analyse the GA results

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

%matplotlib widget

In [4]:
import os
import numpy as np
import pandas as pd
import logging
import pickle
import jupyter_black
from tqdm import tqdm

from src.data_connectors import write_solution_files
from src.data_connectors import read_input_files
from src.data_connectors import read_solutions_files
from src.genetic_algorithm import constants

jupyter_black.load()

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

## Read the results

In [55]:
# filename = "../data/solutions/all_ga_MMTSP_SAC_swap.pkl"
# filename = "../data/solutions/all_ga_MMTSP_SAC_sorted_paretns.pkl"
filename = "../data/solutions/all_ga_MMTSP_SAC_best.pkl"
filename = "../data/solutions/all_ga_MMTSP_SAC.pkl"
with open(filename, "rb") as f:
    loaded_results = pickle.load(f)

In [56]:
len(loaded_results)

27

In [57]:
# First instance
loaded_results[0][-1]

217

In [58]:
# Last instance
loaded_results[-1][-1]

283

### Get Latency results

In [59]:
latencies = []
for instance_number in constants.INSTANCES_LIST:
    latency = read_solutions_files.read_latency_from_solutions_header(
        filename=f"../data/solutions/MMTSP-SAC/solution_{instance_number}.txt"
    )
    latencies.append(latency)

## Process results

list[all_surviving_chromosomes, all_surviving_makespans, best_makespan, duration_until_best_makespan]

In [61]:
instances_path = "../data/input/HRTInstances/"
instances_list = constants.INSTANCES_LIST

times_of_best_results = []

for best, instance_number in tqdm(zip(loaded_results, instances_list)):
    ins_x = ins_x = read_input_files.read_file(
        os.path.join(instances_path, f"Instance_{instance_number}.txt")
    )
    times_of_best_results.append(write_solution_files.find_times_of_best_solution(ins_x, best))

27it [00:03,  8.24it/s]


In [62]:
len(times_of_best_results)

27

In [63]:
times_of_best_results[0]

{'0': {2: (-1, 34),
  22: (34, 69),
  3: (69, 74),
  23: (74, 79),
  4: (79, 109),
  24: (109, 139),
  5: (139, 238),
  25: (238, 337),
  6: (337, 373),
  26: (373, 409),
  7: (409, 450),
  27: (450, 491),
  9: (491, 511),
  29: (511, 531),
  10: (531, 629),
  30: (629, 727),
  12: (727, 771),
  32: (771, 815),
  13: (815, 862),
  33: (862, 909),
  14: (909, 946),
  34: (946, 983),
  15: (983, 1044),
  35: (1044, 1105),
  17: (1105, 1135),
  37: (1135, 1165),
  18: (1165, 1192),
  38: (1192, 1219),
  20: (1219, 1307),
  40: (1307, 1395)},
 '1': {1: (-1, 0), 8: (34, 44), 11: (44, 49), 16: (771, 776), 19: (776, 870)},
 '2': {21: (-1, 0),
  28: (69, 79),
  31: (79, 84),
  36: (815, 820),
  39: (820, 914)}}

## Optimal Solutions

In [64]:
df_optimal = pd.read_csv("../data/solutions/optimal/OptimalSolutions.csv", sep=";", header=1)

df_optimal.head()

,Instance,Humans,Robots,Robot Eligibility,Optimal solution,Best known solution,Lower bound
0,217,1,2,"0,25",1397.0,1397,1397
1,218,1,2,"0,5",865.0,865,865
2,219,1,2,1,529.0,529,529
3,220,1,2,"0,25",3316.0,3316,3316
4,221,1,2,"0,5",2476.0,2476,2476


In [65]:
def get_gap(df_optimal: pd.DataFrame, instance_number: int, best: tuple) -> float:
    lower_bound = df_optimal[df_optimal.Instance == instance_number]["Lower bound"].values[0]
    ins_result = best[2]
    gap = (ins_result - lower_bound) / lower_bound * 100
    logging.debug(f"Lower Bound: {lower_bound}, Result {ins_result}")
    logging.debug(f"Instance {instance_number} has Gap % = {gap:.2f} %")
    return gap

In [66]:
instances_list = [x for x in range(217, 336)]

pct_deviation_from_best = []

for best in tqdm(loaded_results):
    instance_number = best[-1]
    optimal = df_optimal[df_optimal.Instance == instance_number]["Optimal solution"].values[0]
    best_know = df_optimal[df_optimal.Instance == instance_number]["Best known solution"].values[0]
    logging.debug(f"Optimal: {optimal}, Best Known: {best_know}")
    gap = get_gap(df_optimal, instance_number, best)
    logging.debug(f"Instance {instance_number} has Gap % = {gap:.2f} %")

100%|██████████| 27/27 [00:00<00:00, 2902.56it/s]


## Group by type

In [87]:
df_instance_groups = (
    df_optimal.groupby(["Humans", "Robots", "Robot Eligibility"]).Instance.apply(list).reset_index()
)
df_instance_groups


all_gaps = []
all_gaps_sequence = []
all_is_optimal = []
all_lower_gaps = []
all_ids_lower_gaps = []
for idx, row_group in tqdm(df_instance_groups.iterrows()):
    gap_group = []
    is_optimal_group = []
    lower_gaps = []
    ids_lower_gaps = []
    for best in tqdm(loaded_results):
        instance_number = best[-1]
        if instance_number in row_group.Instance:
            gap_x = get_gap(df_optimal, instance_number, best)
            gap_group.append(gap_x)
            if gap_x <= 0:
                is_optimal_group.append(1)
            else:
                is_optimal_group.append(0)
            if gap_x <= 10:
                lower_gaps.append(1)
                ids_lower_gaps.append(instance_number)
            else:
                lower_gaps.append(0)
                ids_lower_gaps.append(0)

    all_gaps.append(gap_group)
    all_gaps_sequence.extend(gap_group)
    all_is_optimal.append(is_optimal_group)
    all_lower_gaps.append(lower_gaps)
    all_ids_lower_gaps.extend(ids_lower_gaps)

# df_instance_groups["Gaps"] = all_gaps
# df_instance_groups["optimals"] = all_is_optimal
# df_instance_groups["lower_gaps"] = all_lower_gaps
# df_instance_groups["Gaps%"] = round(df_instance_groups["Gaps"].apply(lambda x: sum(x) / len(x)), 2)
# df_instance_groups["# optimals"] = df_instance_groups["optimals"].apply(lambda x: sum(x))
# df_instance_groups["# all_lower_gaps"] = df_instance_groups["lower_gaps"].apply(lambda x: sum(x))

100%|██████████| 27/27 [00:00<00:00, 94450.55it/s]
12it [00:00, 171.67it/s]


In [88]:
all_is_optimal

[[1, 1],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0],
 [],
 [0, 0],
 [0],
 [],
 [0],
 [],
 []]

In [89]:
np.mean(all_gaps)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (12,) + inhomogeneous part.

In [82]:
np.std(all_gaps)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (12,) + inhomogeneous part.

In [83]:
df_instance_groups

,Humans,Robots,Robot Eligibility,Instance
0,1,2,"0,25","[217, 220, 229, 232, 241, 244, 253, 256, 265, ..."
1,1,2,"0,5","[218, 221, 230, 233, 242, 245, 254, 257, 266, ..."
2,1,2,1,"[219, 222, 231, 234, 243, 246, 255, 258, 267, ..."
3,1,3,"0,25","[223, 226, 235, 238, 247, 250, 259, 262, 271, ..."
4,1,3,"0,5","[224, 227, 236, 239, 248, 251, 260, 263, 272, ..."
5,1,3,1,"[225, 228, 237, 240, 249, 252, 261, 264, 273, ..."
6,2,2,"0,25","[277, 280, 289, 292, 301, 304, 313, 316, 325, ..."
7,2,2,"0,5","[278, 281, 290, 293, 302, 305, 314, 317, 326, ..."
8,2,2,1,"[279, 282, 291, 294, 303, 306, 315, 318, 327, ..."
9,2,3,"0,25","[283, 286, 295, 298, 307, 310, 319, 322, 331, ..."


In [84]:
df_instance_groups["# all_lower_gaps"].sum()

KeyError: '# all_lower_gaps'

In [85]:
np.mean(latencies)

104.26148148148148

In [72]:
np.mean(latencies) / 60

1.7376913580246913

In [73]:
np.sum(latencies) / 60 / 60

0.7819611111111111

In [74]:
len([x for x in latencies if x < 60])

10

In [75]:
len(latencies)

27

In [76]:
20 / 120

0.16666666666666666

In [77]:
all_lower_gaps_than_10 = [x for x, y in zip(all_gaps_sequence, all_ids_lower_gaps) if y != 0]
all_ids_lower_gaps

[217,
 220,
 218,
 221,
 230,
 233,
 242,
 0,
 266,
 231,
 267,
 223,
 226,
 235,
 238,
 250,
 262,
 271,
 274,
 224,
 227,
 236,
 275,
 277,
 0,
 278,
 283]

In [78]:
len(all_ids_lower_gaps)

27

In [79]:
np.mean(all_lower_gaps_than_10)

4.002426193221288